In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog

from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models.distillation import TrayColumn
from idaes.generic_models.unit_models.distillation.condenser \
    import CondenserType, TemperatureSpec

In [2]:
from HFCS_bmimPF6_PR import configuration

In [3]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

2021-04-29 10:52:38 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [4]:
m.fs.unit = TrayColumn(default={
                       "number_of_trays": 28,
                       "feed_tray_location": 21,
                       "condenser_type": CondenserType.partialCondenser,
                       "condenser_temperature_spec":
                           TemperatureSpec.customTemperature,
                       "property_package": m.fs.props,
                       "has_heat_transfer": False,
                       "has_pressure_change": False})

In [5]:
m.fs.props.PR_kappa['R32', 'R125'].fix(-0.002535)
m.fs.props.PR_kappa['R125', 'R32'].fix(-0.002535)
# m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.01435)
# m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(0.9651)
# m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(0.6312)
# m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.0504)

m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.05102)
m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(0.2839)
m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(1.6929)
m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.04738)

In [6]:
# Fix Heater H101 inlet conditions
m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.20)
m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.10)
m.fs.unit.feed.mole_frac_comp[0, "bmimPF6"].fix(0.7)

m.fs.unit.feed.flow_mol.fix(1)
m.fs.unit.feed.temperature.fix(301)
m.fs.unit.feed.pressure.fix(1000000)

In [7]:
m.fs.unit.condenser.reflux_ratio.fix(0.25)

m.fs.unit.condenser.condenser_pressure.fix(1000000)

m.fs.unit.reboiler.boilup_ratio.fix(1.0)

m.fs.unit.condenser.vapor_outlet.temperature.fix(280)

In [8]:
# print(degrees_of_freedom(m))
# print(degrees_of_freedom(m.fs.unit.condenser))
# print(degrees_of_freedom(m.fs.unit.reboiler))

In [9]:
# m.fs.unit.feed_tray.initialize() 
m.fs.unit.initialize() 
# m.fs.unit.feed_tray.initialize() 

2021-04-29 10:52:44 [INFO] idaes.init.fs.unit: Begin initialization.
2021-04-29 10:52:44 [WARNING] idaes.init.fs.unit: Solver not provided. Default solver(ipopt) being used for initialization.
2021-04-29 10:52:44 [INFO] idaes.init.fs.unit.feed_tray: Begin initialization.
2021-04-29 10:52:44 [WARNING] idaes.init.fs.unit.feed_tray: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-29 10:52:44 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Starting initialization
2021-04-29 10:52:45 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 10:52:45 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Equilibrium temperature initialization completed.
2021-04-29 10:52:46 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-29 10:52:46 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: 

2021-04-29 10:53:17 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Starting initialization
2021-04-29 10:53:18 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 10:53:18 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Equilibrium temperature initialization completed.
2021-04-29 10:53:18 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-29 10:53:19 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-04-29 10:53:19 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Starting initialization
2021-04-29 10:53:20 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Dew and bubble point initialization: infeasible - Converged to a locally infeasi

2021-04-29 10:54:16 [INFO] idaes.init.fs.unit.rectification_section[3].properties_out: Equilibrium temperature initialization completed.
2021-04-29 10:54:17 [INFO] idaes.init.fs.unit.rectification_section[3].properties_out: Phase equilibrium initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:54:18 [INFO] idaes.init.fs.unit.rectification_section[3].properties_out: Property initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:54:18 [INFO] idaes.init.fs.unit.rectification_section[3].properties_out: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:54:21 [INFO] idaes.init.fs.unit.rectification_section[3]: Mass balance solve infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:54:35 [INFO] idaes.init.fs.unit.rectification_section[3]: Mass and energy bal

2021-04-29 10:55:35 [INFO] idaes.init.fs.unit.rectification_section[5]: Mass, energy and pressure balance solve infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:55:35 [INFO] idaes.init.fs.unit.rectification_section[5]: Initialization complete, status infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:55:35 [INFO] idaes.init.fs.unit.rectification_section[6]: Begin initialization.
2021-04-29 10:55:35 [WARNING] idaes.init.fs.unit.rectification_section[6]: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-29 10:55:35 [INFO] idaes.init.fs.unit.rectification_section[6].properties_in_liq: Starting initialization
2021-04-29 10:55:36 [INFO] idaes.init.fs.unit.rectification_section[6].properties_in_liq: Dew and bubble point initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:55:36 [INFO] idaes.init.fs.unit.rectifi

2021-04-29 10:56:25 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_liq: Phase equilibrium initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:56:26 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_liq: Property initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 10:56:26 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_vap: Starting initialization
2021-04-29 10:56:26 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 10:56:26 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_vap: Equilibrium temperature initialization completed.
2021-04-29 10:56:27 [INFO] idaes.init.fs.unit.rectification_section[8].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-29 10:56:27 [INF

...

2021-04-29 11:00:52 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Starting initialization
2021-04-29 11:00:52 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Dew and bubble point initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 11:00:52 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Equilibrium temperature initialization completed.
2021-04-29 11:00:53 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Phase equilibrium initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 11:00:54 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Property initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-29 11:00:54 [INFO] idaes.init.fs.unit.rectification_section[19].properties_out: Property package initialization: infeasible - Converged 

2021-04-29 11:01:48 [INFO] idaes.init.fs.unit.stripping_section[22]: Mass, energy and pressure balance solve optimal - Optimal Solution Found.
2021-04-29 11:01:48 [INFO] idaes.init.fs.unit.stripping_section[22]: Initialization complete, status optimal - Optimal Solution Found.
2021-04-29 11:01:48 [INFO] idaes.init.fs.unit.stripping_section[23]: Begin initialization.
2021-04-29 11:01:48 [WARNING] idaes.init.fs.unit.stripping_section[23]: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-29 11:01:48 [INFO] idaes.init.fs.unit.stripping_section[23].properties_in_liq: Starting initialization
2021-04-29 11:01:51 [INFO] idaes.init.fs.unit.stripping_section[23].properties_in_liq: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 11:01:51 [INFO] idaes.init.fs.unit.stripping_section[23].properties_in_liq: Equilibrium temperature initialization completed.
2021-04-29 11:01:52 [INFO] idaes.init.fs.unit.stripping_section[23].properties

2021-04-29 11:02:25 [INFO] idaes.init.fs.unit.stripping_section[25].properties_out: Equilibrium temperature initialization completed.
2021-04-29 11:02:26 [INFO] idaes.init.fs.unit.stripping_section[25].properties_out: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-29 11:02:27 [INFO] idaes.init.fs.unit.stripping_section[25].properties_out: Property initialization: optimal - Optimal Solution Found.
2021-04-29 11:02:27 [INFO] idaes.init.fs.unit.stripping_section[25].properties_out: Property package initialization: optimal - Optimal Solution Found.
2021-04-29 11:02:30 [INFO] idaes.init.fs.unit.stripping_section[25]: Mass balance solve optimal - Optimal Solution Found.
2021-04-29 11:02:34 [INFO] idaes.init.fs.unit.stripping_section[25]: Mass and energy balance solve optimal - Optimal Solution Found.
2021-04-29 11:02:37 [INFO] idaes.init.fs.unit.stripping_section[25]: Mass, energy and pressure balance solve optimal - Optimal Solution Found.
2021-04-29 11:02:37 [I

2021-04-29 11:03:08 [INFO] idaes.init.fs.unit.stripping_section[28].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 11:03:08 [INFO] idaes.init.fs.unit.stripping_section[28].properties_in_vap: Equilibrium temperature initialization completed.
2021-04-29 11:03:08 [INFO] idaes.init.fs.unit.stripping_section[28].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-29 11:03:09 [INFO] idaes.init.fs.unit.stripping_section[28].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-04-29 11:03:09 [INFO] idaes.init.fs.unit.stripping_section[28].properties_out: Starting initialization
2021-04-29 11:03:09 [INFO] idaes.init.fs.unit.stripping_section[28].properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-29 11:03:09 [INFO] idaes.init.fs.unit.stripping_section[28].properties_out: Equilibrium temperature initialization completed.
2021-04

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
# Import python path
import os

# Import idaes model seralizer to store initialized model
from idaes.core.util import model_serializer as ms

In [ ]:
ms.to_json(m, fname="tray.json.gz")


In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
results = solver.solve(m)

status = solver.solve(m, tee = True)

In [ ]:
from idaes.core.util.model_statistics import large_residuals_set

In [ ]:
print(large_residuals_set(m))

In [ ]:
m.display()